#**<center>LAB-04**

###**Student id:** 202211037

###**Student Name:** Aksh Patel

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=83c7ddf486a7f82a07ba685d5c58fbf7dba7f0f00170e98c644c2e2ed14a651f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Exercise 1

In [ ]:
from pyspark import SparkContext, SparkConf
from tabulate import tabulate

# Create a SparkConf and SparkContext
conf = SparkConf().setAppName("EmployeeCSV")
sc = SparkContext(conf=conf)

# Load the CSV file as an RDD
lines = sc.textFile("/content/gdrive/My Drive/big_data_processing/datasets/mr/employee.csv")

# Filter the RDD to select employees from state 'TX'
tx_employees = lines.filter(lambda line: line.split(",")[4] == "TX")

# Project and print (empno, name) of employees from state 'TX'
tx_empno_name = tx_employees.map(lambda line: (line.split(",")[0], line.split(",")[1]))
results = tx_empno_name.collect()  # Collect the results and print

# Create a tabular representation of the results
table = tabulate(results, headers=["Employee Number", "Name"], tablefmt="grid")

# Print the tabular representation
print(table)

+-------------------+-----------------+
|   Employee Number | Name            |
+===================+=================+
|             10012 | Hector Barbossa |
+-------------------+-----------------+
|             10200 | Anton Chigurh   |
+-------------------+-----------------+
|             10202 | Jac McKinzie    |
+-------------------+-----------------+


## Exercise 2

In [ ]:
from pyspark import SparkConf, SparkContext

# Create the existing SparkContext
sc = SparkContext.getOrCreate()
input =  "/content/gdrive/MyDrive/big_data_processing/datasets/mr/employee.csv"

# Load the CSV file into an RDD
emp_rdd = sc.textFile(input)

employee_data_rdd = emp_rdd \
    .map(lambda line: line.split(",")) \
    .map(lambda fields: (int(fields[0]), fields[1], int(fields[2]), int(fields[3])))

department_avg_salaries = employee_data_rdd \
    .map(lambda data: (data[2], (data[3], 1))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])) \
    .mapValues(lambda x: x[0] / x[1])

broadcast_avg_dep = sc.broadcast(department_avg_salaries.collectAsMap())

high_salary_emp_rdd = employee_data_rdd \
    .filter(lambda data: data[3] > 1.5 * broadcast_avg_dep.value[data[2]])

high_salary_emp = high_salary_emp_rdd.collect()
for emp in high_salary_emp:
    print(f"Emp No: {emp[0]}, Name: {emp[1]}, Salary: {emp[3]}, Department Average Salary: {broadcast_avg_dep.value[emp[2]]}")

sc.stop()

Emp No: 10019, Name: Vito Corleone, Salary: 170500, Department Average Salary: 59967.78846153846
Emp No: 10015, Name: Jason Foss, Salary: 178000, Department Average Salary: 97064.64
Emp No: 10272, Name: Debra Houlihan, Salary: 180000, Department Average Salary: 68684.0625
Emp No: 10288, Name: Peter Monroe, Salary: 157000, Department Average Salary: 97064.64
Emp No: 10086, Name: Katie Roper, Salary: 150290, Department Average Salary: 97064.64
Emp No: 10222, Name: Ricardo Ruiz, Salary: 148999, Department Average Salary: 97064.64
Emp No: 10010, Name: Jennifer Zamora, Salary: 220450, Department Average Salary: 97064.64


## Exercise 3

In [ ]:
# Create the existing SparkContext
sc = SparkContext.getOrCreate()
input =  "/content/gdrive/MyDrive/big_data_processing/datasets/mr/employee.csv"

# Load the CSV file into an RDD
lines = sc.textFile("/content/gdrive/My Drive/big_data_processing/datasets/mr/employee.csv")

# Split each line into fields and extract the state
def extract_state(line):
    fields = line.split(',')
    state = fields[4]
    return (state, 1)

# Use the "map" operation to create key-value pairs with state as key and 1 as value
state_employee_counts = lines.map(extract_state)

# Use the "reduceByKey" operation to count employees by state
state_counts = state_employee_counts.reduceByKey(lambda x, y: x + y)

# Collect the results and print the state-wise employee counts
results = state_counts.collect()

for state, count in results:
    print("State:", state, ", Employee Count:", count)


State: TX , Employee Count: 3
State: CT , Employee Count: 6
State: VA , Employee Count: 1
State: CA , Employee Count: 1
State: OH , Employee Count: 1
State: TN , Employee Count: 1
State: RI , Employee Count: 1
State: CO , Employee Count: 1
State: NY , Employee Count: 1
State: NC , Employee Count: 1
State: ME , Employee Count: 1
State: MA , Employee Count: 276
State: VT , Employee Count: 2
State: AL , Employee Count: 1
State: WA , Employee Count: 1
State: IN , Employee Count: 1
State: NH , Employee Count: 1
State: PA , Employee Count: 1
State: UT , Employee Count: 1
State: GA , Employee Count: 1
State: FL , Employee Count: 1
State: KY , Employee Count: 1
State: ID , Employee Count: 1
State: NV , Employee Count: 1
State: MT , Employee Count: 1
State: OR , Employee Count: 1
State: ND , Employee Count: 1
State: AZ , Employee Count: 1


##Exercise 4

In [ ]:
import math

# Load the CSV file into an RDD
lines = sc.textFile("/content/gdrive/My Drive/big_data_processing/datasets/mr/employee.csv")

# Split each line into fields and extract the salary
def extract_salary(line):
    fields = line.split(',')
    salary = float(fields[3])  # Assuming salary is in the fourth column
    return salary

# Use the "map" operation to extract salary values
salaries = lines.map(extract_salary)

# Calculate the mean and standard deviation
total_salaries = salaries.sum()
count = salaries.count()
mean_salary = total_salaries / count
squared_diff = salaries.map(lambda x: (x - mean_salary) ** 2)
variance = squared_diff.sum() / count
std_deviation = math.sqrt(variance)

# Create a table to display the result
table_data = [["Metric", "Value"],
              ["Mean Salary", mean_salary],
              ["Standard Deviation", std_deviation]]

# Use tabulate to format the data as a table
table = tabulate(table_data, headers="firstrow", tablefmt="pretty")

# Print the table
print(table)


+--------------------+--------------------+
|       Metric       |       Value        |
+--------------------+--------------------+
|    Mean Salary     |  69020.6848874598  |
| Standard Deviation | 25116.159611505253 |
+--------------------+--------------------+


##Exercise 5

In [ ]:
# Split each line into fields and extract department and salary
def extract_department_salary(line):
    fields = line.split(',')
    department = fields[2]  # Assuming department is in the third column
    salary = float(fields[3])  # Assuming salary is in the fourth column
    return (department, salary)

# Use the "map" operation to extract department and salary values
department_salary = lines.map(extract_department_salary)

# Group the data by department
department_groups = department_salary.groupByKey()

# Calculate the standard deviation of salaries for each department
def calculate_std_deviation(values):
    values_list = list(values)
    n = len(values_list)
    if n == 0:
        return 0.0
    mean_salary = sum(values_list) / n
    squared_diff = [(x - mean_salary) ** 2 for x in values_list]
    variance = sum(squared_diff) / n
    std_deviation = math.sqrt(variance)
    return std_deviation

department_std_deviation = department_groups.mapValues(calculate_std_deviation)

# Create a table to display the result
table_data = [["Department", "Salary Standard Deviation"]] + department_std_deviation.collect()

# Use tabulate to format the data as a table
table = tabulate(table_data, headers="firstrow", tablefmt="pretty")

# Print the table
print(table)

+------------+---------------------------+
| Department | Salary Standard Deviation |
+------------+---------------------------+
|     4      |     8754.93245490792      |
|     1      |     19722.68484258672     |
|     5      |    11420.800779947584     |
|     3      |     32875.83877242373     |
|     6      |    20702.734864531638     |
|     2      |            0.0            |
+------------+---------------------------+


##Exercise 6

In [ ]:
import math

# Split each line into fields and extract employee information, department, and salary
def extract_employee_info(line):
    fields = line.split(',')
    empno = fields[0]
    dno = fields[2]
    name = fields[1]
    salary = float(fields[3])  # Assuming salary is in the fourth column
    return (empno, dno, name, salary)

# Use the "map" operation to extract employee details
employee_details = lines.map(extract_employee_info)

# Calculate the standard deviation of salary for each department
department_salaries = employee_details.map(lambda x: (x[1], [x[3]]))
department_std_deviation = department_salaries.reduceByKey(lambda x, y: x + y) \
    .mapValues(lambda salaries: math.sqrt(sum((salary - sum(salaries) / len(salaries)) ** 2 for salary in salaries) / len(salaries)))

# Broadcast the department-wise standard deviation to all workers
department_std_deviation_broadcast = sc.broadcast(dict(department_std_deviation.collect()))

# Filter employees with salaries > 1.5 times the department's standard deviation
filtered_employees = employee_details.filter(lambda x: x[3] > 1.5 * department_std_deviation_broadcast.value[x[1]])

# List (empno, dno, name, salary, dept_sal_sd) for filtered employees
result = filtered_employees.map(lambda x: (x[0], x[1], x[2], x[3], department_std_deviation_broadcast.value[x[1]]))

# Collect and print the results
for emp_info in result.collect():
     print(emp_info)

('10026', '5', 'Wilson K Adinolfi', 62506.0, 11420.800779947584)
('10084', '3', 'Karthikeyan Ait Sidi', 104437.0, 32875.83877242373)
('10196', '5', 'Sarah Akinkuolie', 64955.0, 11420.800779947584)
('10088', '5', 'Trina Alagbe', 64991.0, 11420.800779947584)
('10069', '5', 'Carol Anderson', 50825.0, 11420.800779947584)
('10002', '5', 'Linda Anderson', 57568.0, 11420.800779947584)
('10194', '4', 'Colby Andreola', 95660.0, 8754.93245490792)
('10062', '5', 'Sam Athwal', 59365.0, 11420.800779947584)
('10114', '5', 'Linda Bachiochi', 47837.0, 11420.800779947584)
('10250', '3', 'Alejandro Bacong', 50178.0, 32875.83877242373)
('10252', '5', 'Rachael Baczenski', 54670.0, 11420.800779947584)
('10242', '5', 'Thomas Barbara', 47211.0, 11420.800779947584)
('10012', '3', 'Hector Barbossa', 92328.0, 32875.83877242373)
('10265', '5', 'Francesco A Barone', 58709.0, 11420.800779947584)
('10066', '5', 'Nader Barton', 52505.0, 11420.800779947584)
('10061', '5', 'Norman Bates', 57834.0, 11420.800779947584)


##Exercise 7

In [ ]:
input = "/content/gdrive/My Drive/big_data_processing/datasets/mr/employee.csv"

# Load the CSV file into an RDD
emp_rdd = sc.textFile(input)

employee_data_rdd = emp_rdd \
    .map(lambda line: line.split(",")) \
    .map(lambda fields: (int(fields[0]), fields[1], int(fields[2]), int(fields[3])))

mean_sal = employee_data_rdd \
    .map(lambda data: data[3]) \
    .mean()

department_stats_rdd = employee_data_rdd \
    .map(lambda data: (data[2], (data[3], 1))) \
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

department_offsets_rdd = department_stats_rdd \
    .mapValues(lambda stats: stats[0] / stats[1] - mean_sal)

department_offsets = department_offsets_rdd.collect()

for dep_num, offset in department_offsets:
    print(f"Department {dep_num} salary offset from overall average: {offset}")

Department 4 salary offset from overall average: 27131.4151125402
Department 6 salary offset from overall average: -336.6223874598072
Department 2 salary offset from overall average: 180979.3151125402
Department 5 salary offset from overall average: -9052.896425921346
Department 3 salary offset from overall average: 28043.955112540192
Department 1 salary offset from overall average: 3928.315112540193


##Exercise 8

In [ ]:
sc.stop()

from pyspark import SparkConf, SparkContext
import re

conf = SparkConf().setAppName("WebAccessLogSummary")
sc = SparkContext(conf=conf)

log_file_path = "/content/gdrive/MyDrive/big_data_processing/datasets/mr/web_access_log.txt"
log_rdd = sc.textFile(log_file_path)

date_pattern = r'\[(\d{2}/[A-Za-z]{3}/\d{4}:\d{2}:\d{2}:\d{2} [+\-]\d{4})\]'

def extract_month(line):
    match = re.search(date_pattern, line)
    if match:
        date_str = match.group(1)
        return date_str.split('/')[1]
    else:
        return None

month_rdd = log_rdd.map(extract_month)

valid_month_rdd = month_rdd.filter(lambda x: x is not None)

requests_by_month = valid_month_rdd.countByValue()

for month, count in requests_by_month.items():
    print(f"Month: {month}, Total Number of Requests: {count}")


Month: Dec, Total Number of Requests: 14148
Month: Jan, Total Number of Requests: 28224
Month: Feb, Total Number of Requests: 64262


##Exercise 9

In [ ]:
log_pattern = r'(\S+) \S+ \S+ \[([^\]]+)\] "(\S+) \S+ \S+" (\d+) \d+'

def extract_timestamp_url(line):
    match = re.search(log_pattern, line)
    if match:
        timestamp = match.group(2)
        url = match.group(3)
        status_code = int(match.group(4))
        return (timestamp, url, status_code)
    else:
        return None

timestamp_url_status_rdd = log_rdd.map(extract_timestamp_url)

not_found_requests_rdd = timestamp_url_status_rdd.filter(lambda x: x is not None and x[2] == 404)

results = not_found_requests_rdd.collect()

for timestamp, url, status_code in results:
    print(f"Timestamp: {timestamp}, URL: {url}, Status Code: {status_code}")

sc.stop()

Timestamp: 12/Dec/2015:19:02:36 +0100, URL: GET, Status Code: 404
Timestamp: 12/Dec/2015:19:44:06 +0100, URL: GET, Status Code: 404
Timestamp: 12/Dec/2015:19:44:15 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:01:01:19 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:11:28:41 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:12:05:25 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:15:56:36 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:16:14:58 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:18:29:14 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:18:29:18 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:18:39:08 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:20:24:02 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:20:27:28 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:21:04:08 +0100, URL: GET, Status Code: 404
Timestamp: 13/Dec/2015:21:04:09 +0100, URL: GET, Status Code: 404
Timestamp:

##**Colab Notebook link:**

[202211037_BDP_lab04.ipynb](https://colab.research.google.com/drive/1kakZ1ajIp4r74K4MhH87dsB-xF4YttfF#scrollTo=7WnCBDS5U6e0)

In [ ]:
!python --version

Python 3.10.12
